Python Code Overview
The Python code in this Jupyter notebook builds and places components in the divide-by-2 circuit using the sky130 PDK and gdsfactory layout library.

create_nmos_latch Function: Defines a single NMOS-based latch with eight NMOS transistors. Transistors are arranged in the current mirror, transmission gate, and cross-coupled inverter configurations. The function:

Creates NMOS transistors with specific widths and lengths.
Positions and connects these transistors using both straight and C-routes for proper wiring.
Adds ports for input (D, Dp), clock (CLK, CLKN), output (Q, Qp), and power (VDD, VSS).
create_divide_by_two_circuit Function: Builds the entire divide-by-2 circuit by instantiating two NMOS latches.

Places the second latch below the first latch with adequate separation.
Connects the output of the first latch to the input of the second and vice versa (feedback).
Adds ports for the clock (CLK, CLKN), output (OUT, OUT_B), and power (VDD, VSS).
Connects power ports of both latches to ensure consistent power distribution.
Circuit Display and GDS Export:

After creating the divide-by-two component, display_component shows a scaled view of the component.
The component is written to a GDS file, divide_two.gds, for further physical verification and layout.

In [ ]:
# Setup the environment for the OpenFASOC GDSFactory generator
# You only need to run this block once!

# Clone OpenFASoC
!git clone https://github.com/idea-fasoc/OpenFASOC
# Install python dependencies
!pip install sky130
!pip install gf180 prettyprinttree svgutils
!pip install gdsfactory==7.7.0

import pathlib
import os
# Install KLayout (via conda)
!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
%env CONDA_PREFIX={CONDA_PREFIX}

!bin/micromamba create --yes --prefix $CONDA_PREFIX
# Install from the litex-hub channel
!bin/micromamba install --yes --prefix $CONDA_PREFIX \
                        --channel litex-hub \
                        --channel main \
                        klayout

In [ ]:
# Setup the environment for the OpenFASOC GDSFactory generator

# Adding micro-mamba binary directory to the PATH
# This directory contains Klayout
import pathlib
import os
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
%env CONDA_PREFIX={CONDA_PREFIX}
# Add conda packages to the PATH
PATH = os.environ['PATH']
%env PATH={PATH}:{CONDA_PREFIX}/bin

%cd /content/OpenFASOC/openfasoc/generators/glayout

In [ ]:
%cd /content/OpenFASOC/openfasoc/generators/glayout
!pip install -e .

In [ ]:
!pip show glayout

In [ ]:
import glayout

In [ ]:
import gdstk
import svgutils.transform as sg
import IPython.display
from IPython.display import clear_output
import ipywidgets as widgets

# Redirect all outputs here
hide = widgets.Output()

def display_gds(gds_file, scale = 3):
  # Generate an SVG image
  top_level_cell = gdstk.read_gds(gds_file).top_level()[0]
  top_level_cell.write_svg('out.svg')
  # Scale the image for displaying
  fig = sg.fromfile('out.svg')
  fig.set_size((str(float(fig.width) * scale), str(float(fig.height) * scale)))
  fig.save('out.svg')

  # Display the image
  IPython.display.display(IPython.display.SVG('out.svg'))

def display_component(component, scale = 3):
  # Save to a GDS file
  with hide:
    component.write_gds("out.gds")
  display_gds('out.gds', scale)


In [ ]:
#Latch template
from glayout.flow.pdk.sky130_mapped import sky130_mapped_pdk as sky130
from gdsfactory import Component
from glayout.flow.primitives.fet import nmos
from glayout.flow.routing.straight_route import straight_route
from glayout.flow.routing.c_route import c_route

def create_nmos_latch_layout(pdk):
    latch = Component("nmos_latch")

    # Create NMOS transistors
    m2 = latch << nmos(pdk, width=1, length=0.15)
    m7 = latch << nmos(pdk, width=1, length=0.15)
    m3 = latch << nmos(pdk, width=2, length=0.15)
    m4 = latch << nmos(pdk, width=2, length=0.15)
    m5 = latch << nmos(pdk, width=2, length=0.15)
    m6 = latch << nmos(pdk, width=2, length=0.15)
    m8 = latch << nmos(pdk, width=2, length=0.15)

    # Position transistors
    spacing = pdk.util_max_metal_seperation() + m2.size[0]
    m7.movex(6 * spacing)
    m3.movex(2 * spacing)
    m4.movex(3 * spacing)
    m5.movex(4 * spacing)
    m6.movex(5 * spacing)
    m8.movex(spacing)

    # Routing
    # Connect transmission gates
    latch << straight_route(pdk, m2.ports["multiplier_0_drain_E"], m8.ports["multiplier_0_source_E"])
    latch << straight_route(pdk, m7.ports["multiplier_0_drain_E"], m6.ports["multiplier_0_source_E"])

    # Connect current mirror
    latch << straight_route(pdk, m3.ports["multiplier_0_drain_E"], m4.ports["multiplier_0_drain_E"])
    latch << straight_route(pdk, m4.ports["multiplier_0_drain_E"], m5.ports["multiplier_0_drain_E"])
    latch << straight_route(pdk, m3.ports["multiplier_0_gate_E"], m4.ports["multiplier_0_gate_E"])
    latch << straight_route(pdk, m4.ports["multiplier_0_gate_E"], m5.ports["multiplier_0_gate_E"])

    # Connect cross-coupled inverters
    latch << c_route(pdk, m6.ports["multiplier_0_drain_E"], m8.ports["multiplier_0_gate_E"])
    latch << c_route(pdk, m8.ports["multiplier_0_drain_E"], m6.ports["multiplier_0_gate_E"])

    # Connect to current mirror
    latch << straight_route(pdk, m6.ports["multiplier_0_source_E"], m5.ports["multiplier_0_drain_E"])
    latch << straight_route(pdk, m8.ports["multiplier_0_source_E"], m5.ports["multiplier_0_drain_E"])

    # Add ports
    latch.add_port("D", port=m2.ports["multiplier_0_source_E"])
    latch.add_port("Dp", port=m7.ports["multiplier_0_source_E"])
    latch.add_port("CLK", port=m2.ports["multiplier_0_gate_E"])
    latch.add_port("CLKN", port=m7.ports["multiplier_0_gate_E"])
    latch.add_port("Q", port=m6.ports["multiplier_0_drain_E"])
    latch.add_port("Qp", port=m8.ports["multiplier_0_drain_E"])
    latch.add_port("VDD", port=m3.ports["multiplier_0_drain_E"])
    latch.add_port("VSS", port=m5.ports["multiplier_0_source_E"])

    return latch

# Create the NMOS-only latch layout using sky130 PDK
nmos_latch = create_nmos_latch_layout(sky130)
display_component(nmos_latch, scale=3)
nmos_latch.write_gds("nmos_latch_layout_corrected_ports.gds")

In [ ]:
#divider template
from glayout.flow.pdk.sky130_mapped import sky130_mapped_pdk as sky130
from gdsfactory import Component
from glayout.flow.primitives.fet import nmos
from glayout.flow.routing.straight_route import straight_route
from glayout.flow.routing.c_route import c_route

def create_nmos_latch(pdk):
    latch = Component("nmos_latch")

    # Create NMOS transistors
    m2 = latch << nmos(pdk, width=1, length=0.15)
    m7 = latch << nmos(pdk, width=1, length=0.15)
    m3 = latch << nmos(pdk, width=2, length=0.15)
    m4 = latch << nmos(pdk, width=2, length=0.15)
    m5 = latch << nmos(pdk, width=2, length=0.15)
    m6 = latch << nmos(pdk, width=2, length=0.15)
    m8 = latch << nmos(pdk, width=2, length=0.15)

    # Position transistors
    spacing = pdk.util_max_metal_seperation() + m2.size[0]
    m7.movex(6 * spacing)
    m3.movex(2 * spacing)
    m4.movex(3 * spacing)
    m5.movex(4 * spacing)
    m6.movex(5 * spacing)
    m8.movex(spacing)

    # Routing
    # Connect transmission gates
    latch << straight_route(pdk, m2.ports["multiplier_0_drain_E"], m8.ports["multiplier_0_source_E"])
    latch << straight_route(pdk, m7.ports["multiplier_0_drain_E"], m6.ports["multiplier_0_source_E"])

    # Connect current mirror
    latch << straight_route(pdk, m3.ports["multiplier_0_drain_E"], m4.ports["multiplier_0_drain_E"])
    latch << straight_route(pdk, m4.ports["multiplier_0_drain_E"], m5.ports["multiplier_0_drain_E"])
    latch << straight_route(pdk, m3.ports["multiplier_0_gate_E"], m4.ports["multiplier_0_gate_E"])
    latch << straight_route(pdk, m4.ports["multiplier_0_gate_E"], m5.ports["multiplier_0_gate_E"])

    # Connect cross-coupled inverters
    latch << c_route(pdk, m6.ports["multiplier_0_drain_E"], m8.ports["multiplier_0_gate_E"])
    latch << c_route(pdk, m8.ports["multiplier_0_drain_E"], m6.ports["multiplier_0_gate_E"])

    # Connect to current mirror
    latch << straight_route(pdk, m6.ports["multiplier_0_source_E"], m5.ports["multiplier_0_drain_E"])
    latch << straight_route(pdk, m8.ports["multiplier_0_source_E"], m5.ports["multiplier_0_drain_E"])

    # Add ports
    latch.add_port("D", port=m2.ports["multiplier_0_source_E"])
    latch.add_port("Dp", port=m7.ports["multiplier_0_source_E"])
    latch.add_port("CLK", port=m2.ports["multiplier_0_gate_E"])
    latch.add_port("CLKN", port=m7.ports["multiplier_0_gate_E"])
    latch.add_port("Q", port=m6.ports["multiplier_0_drain_E"])
    latch.add_port("Qp", port=m8.ports["multiplier_0_drain_E"])
    latch.add_port("VDD", port=m3.ports["multiplier_0_drain_E"])
    latch.add_port("VSS", port=m5.ports["multiplier_0_source_E"])

    return latch

def create_divide_by_two_circuit(pdk):
    divider = Component("divide_by_two")

    # Create two latches
    latch1 = divider << create_nmos_latch(pdk)
    latch2 = divider << create_nmos_latch(pdk)

    # Position second latch below the first
    latch2.movey(-latch1.size[1] - pdk.util_max_metal_seperation())

    # Connect latches
    divider << straight_route(pdk, latch1.ports["Q"], latch2.ports["D"])
    divider << straight_route(pdk, latch1.ports["Qp"], latch2.ports["Dp"])

    # Feedback connection
    divider << c_route(pdk, latch2.ports["Q"], latch1.ports["D"])
    divider << c_route(pdk, latch2.ports["Qp"], latch1.ports["Dp"])

    # Add circuit ports
    divider.add_port("CLK", port=latch1.ports["CLK"])
    divider.add_port("CLKN", port=latch1.ports["CLKN"])
    divider.add_port("OUT", port=latch2.ports["Q"])
    divider.add_port("OUT_B", port=latch2.ports["Qp"])
    divider.add_port("VDD", port=latch1.ports["VDD"])
    divider.add_port("VSS", port=latch1.ports["VSS"])

    # Connect VDD and VSS of both latches
    divider << straight_route(pdk, latch1.ports["VDD"], latch2.ports["VDD"])
    divider << straight_route(pdk, latch1.ports["VSS"], latch2.ports["VSS"])

    return divider

# Create the NMOS-only latch layout using sky130 PDK
divide_two = create_divide_by_two_circuit(sky130)
display_component(divide_two, scale=3)
divide_two.write_gds("divide_two.gds")